# PySnooper - Never use print for debugging again 
[**GitHub Documentation**](https://github.com/cool-RR/PySnooper)

PySnooper is a poor man's debugger. If you've used Bash, it's like set -x for Python, except it's fancier.

Your story: You're trying to figure out why your Python code isn't doing what you think it should be doing. You'd love to use a full-fledged debugger with breakpoints and watches, but you can't be bothered to set one up right now.

You want to know which lines are running and which aren't, and what the values of the local variables are.

Most people would use print lines, in strategic locations, some of them showing the values of variables.

PySnooper lets you do the same, except instead of carefully crafting the right print lines, you just add one decorator line to the function you're interested in. You'll get a play-by-play log of your function, including which lines ran and when, and exactly when local variables were changed.

What makes PySnooper stand out from all other code intelligence tools? You can use it in your shitty, sprawling enterprise codebase without having to do any setup. Just slap the decorator on, as shown below, and redirect the output to a dedicated log file by specifying its path as the first argument.

## Installation

**Installation with Pip**

The best way to install **PySnooper** is with Pip:

```console
$ pip install pysnooper
```

**Other installation options**

Conda with conda-forge channel:

```console
$ conda install -c conda-forge pysnooper
```


In [1]:
!pip install pysnooper

## Basic Example

We're writing a function that converts a number to binary, by returning a list of bits. Let's snoop on it by adding the `@pysnooper.snoop()` decorator:

In [2]:
import pysnooper

@pysnooper.snoop()
def number_to_bits(number):
    if number:
        bits = []
        while number:
            number, remainder = divmod(number, 2)
            bits.insert(0, remainder)
        return bits
    else:
        return [0]

number_to_bits(6)


PYDEV DEBUGGER WARNING:
sys.settrace() should not be used when the debugger is being used.
This may cause the debugger to stop working correctly.
If this is needed, please check: 
http://pydev.blogspot.com/2007/06/why-cant-pydev-debugger-work-with.html
to see how to restore the debug tracing back correctly.
Call Location:
  File "/usr/local/lib/python3.7/dist-packages/pysnooper/tracer.py", line 340, in __enter__
    sys.settrace(self.trace)

Source path:... <ipython-input-2-d9086aeb58cb>
Starting var:.. number = 6
09:07:16.413251 call         4 def number_to_bits(number):
09:07:16.414475 line         5     if number:
09:07:16.414781 line         6         bits = []
New var:....... bits = []
09:07:16.415055 line         7         while number:
09:07:16.415537 line         8             number, remainder = divmod(number, 2)
Modified var:.. number = 3
New var:....... remainder = 0
09:07:16.415824 line         9             bits.insert(0, remainder)
Modified var:.. bits = [0]
09:07:16.416

[1, 1, 0]

## More examples

When calling the functions, PySnooper is also able to clearly show the hierachical relationships between two or more functoins. 

In [3]:
@pysnooper.snoop()
def two(x, y):
    z = x + y
    return z
    
@pysnooper.snoop()
def one(number):
    k = 0
    while number:
        k = two(k, number)
        number -= 1
    return number

In [4]:
one(3)

Source path:... <ipython-input-3-94a80f6770e5>
Starting var:.. number = 3
09:07:16.445779 call         7 def one(number):
09:07:16.447035 line         8     k = 0
New var:....... k = 0
09:07:16.447346 line         9     while number:
09:07:16.447920 line        10         k = two(k, number)
    Source path:... <ipython-input-3-94a80f6770e5>
    Starting var:.. x = 0
    Starting var:.. y = 3
    09:07:16.448436 call         2 def two(x, y):
    09:07:16.449426 line         3     z = x + y
    New var:....... z = 3
    09:07:16.449715 line         4     return z
    09:07:16.450289 return       4     return z
    Return value:.. 3
    Elapsed time: 00:00:00.002741
Modified var:.. k = 3
09:07:16.451594 line        11         number -= 1
Modified var:.. number = 2
09:07:16.453997 line         9     while number:
09:07:16.454807 line        10         k = two(k, number)
    Starting var:.. x = 3
    Starting var:.. y = 2
    09:07:16.455340 call         2 def two(x, y):
    09:07:16.456137

0

# Some Other Features (Optional)#

If stderr is not easily accessible for you, you can redirect the output to a file:

```python
@pysnooper.snoop('/my/log/file.log')
```

You can also pass a stream or a callable instead, and they'll be used.

See values of some expressions that aren't local variables:

```python
@pysnooper.snoop(watch=('foo.bar', 'self.x["whatever"]'))
```

Show snoop lines for functions that your function calls:

```python
@pysnooper.snoop(depth=2)
```

**See [Advanced Usage](https://github.com/cool-RR/PySnooper/blob/master/ADVANCED_USAGE.md) for more options.** 

# Advanced Usage [Optional!] 

Use `watch_explode` to expand values to see all their attributes or items of lists/dictionaries:

```python
@pysnooper.snoop(watch_explode=('foo', 'self'))
```

`watch_explode` will automatically guess how to expand the expression passed to it based on its class. You can be more specific by using one of the following classes:

```python
import pysnooper

@pysnooper.snoop(watch=(
    pysnooper.Attrs('x'),    # attributes
    pysnooper.Keys('y'),     # mapping (e.g. dict) items
    pysnooper.Indices('z'),  # sequence (e.g. list/tuple) items
))
```

Exclude specific keys/attributes/indices with the `exclude` parameter, e.g. `Attrs('x', exclude=('_foo', '_bar'))`.

Add a slice after `Indices` to only see the values within that slice, e.g. `Indices('z')[-3:]`.

```console
$ export PYSNOOPER_DISABLED=1 # This makes PySnooper not do any snooping
```

This will output lines like:

```
Modified var:.. foo[2] = 'whatever'
New var:....... self.baz = 8
```

Start all snoop lines with a prefix, to grep for them easily:

```python
@pysnooper.snoop(prefix='ZZZ ')
```

Remove all machine-related data (paths, timestamps, memory addresses) to compare with other traces easily:

```python
@pysnooper.snoop(normalize=True)
```

On multi-threaded apps identify which thread are snooped in output:

```python
@pysnooper.snoop(thread_info=True)
```

PySnooper supports decorating generators.

If you decorate a class with `snoop`, it'll automatically apply the decorator to all the methods. (Not including properties and other special cases.)

You can also customize the repr of an object:




In [5]:
import numpy
def large(l):
    return isinstance(l, list) and len(l) > 5
 
def print_list_size(l):
    return 'list(size={})'.format(len(l))
 
def print_ndarray(a):
    return 'ndarray(shape={}, dtype={})'.format(a.shape, a.dtype)
 
@pysnooper.snoop(custom_repr=((large, print_list_size), (numpy.ndarray, print_ndarray)))
def sum_to_x(x):
    l = list(range(x))
    a = numpy.zeros((10,10))
    return sum(l)
 
sum_to_x(10000)

Source path:... <ipython-input-5-0347b903ac08>
Starting var:.. x = 10000
09:07:16.482398 call        12 def sum_to_x(x):
09:07:16.484122 line        13     l = list(range(x))
New var:....... l = list(size=10000)
09:07:16.484905 line        14     a = numpy.zeros((10,10))
New var:....... a = ndarray(shape=(10, 10), dtype=float64)
09:07:16.485288 line        15     return sum(l)
09:07:16.485762 return      15     return sum(l)
Return value:.. 49995000
Elapsed time: 00:00:00.004114


49995000

You will get `l = list(size=10000)` for the list, and `a = ndarray(shape=(10, 10), dtype=float64)` for the ndarray.
The `custom_repr` are matched in order, if one condition matches, no further conditions will be checked.



Variables and exceptions get truncated to 100 characters by default. You
can customize that:

```python
    @pysnooper.snoop(max_variable_length=200)
```

You can also use `max_variable_length=None` to never truncate them.

Use `relative_time=True` to show timestamps relative to start time rather than
wall time.